In [2]:
%load_ext autoreload
%autoreload 2
import uproot
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import sys
import importlib
import coffea.util as util
import time
import json
import os
import numba as nb
import awkward.numba

sys.path.append("../../analysisTools/")
from analysisTools import Analyzer
from analysisTools import loadSchema
import analysisTools as tools
import analysisSubroutines as routines

In [3]:
events = tools.loadNano("root://cmseos.fnal.gov//store/group/lpcmetx/iDMe//Samples/Ntuples/signal_v2/2018/Mchi-22p0_dMchi-4p0/ctau-10//ntuples_Mchi-22p0_dMchi-4p0_ctau-10_00.root")

In [4]:
#################################
#### Hard-coded basic cuts ######
#################################
# 1 or 2 jets in the event
nJets = ak.count(events.PFJet.pt,axis=1)
events = events[(nJets>0) & (nJets<3)]

#################################
## Calculating Additional Vars ##
#################################
routines.electronJetSeparation(events) # dR and dPhi between electrons and jets
routines.electronIsoConePtSum(events) # for each electron, compute pT sum of any other electrons in event within dR < 0.3 of it
routines.electronID(events) # electron kinematic/ID definition
routines.vtxElectronConnection(events) # associate electrons to vertices
routines.LxyID(events) # ID cut for low pT electrons in vertices with Lxy > 0.5
routines.defineGoodVertices(events) # define "good" vertices based on whether associated electrons pass ID cuts

In [5]:
i=7052

In [349]:
events.vtx.e1.IDscore[i]

<Array [1, -0.25, -0.25] type='3 * float32'>

In [350]:
events.vtx.e1_typ[i]

<Array ['R', 'L', 'L'] type='3 * string'>

In [351]:
events.vtx.e2.IDscore[i]

<Array [1, 1, 1] type='3 * float32'>

In [352]:
events.vtx.e2_typ[i]

<Array ['R', 'R', 'R'] type='3 * string'>

In [353]:
events.vtx.vxy[i]

<Array [0.0428, 0.462, 0.587] type='3 * float32[parameters={"__doc__": "vtx_vxy"}]'>

In [354]:
events.vtx.passLxyID1[i]

<Array [True, False, True] type='3 * bool'>

In [355]:
events.vtx.passLxyID2[i]

<Array [True, True, True] type='3 * bool'>

In [356]:
events.vtx.e1.passID[i]

<Array [False, True, True] type='3 * bool'>

In [357]:
events.vtx.e2.passID[i]

<Array [True, False, True] type='3 * bool'>

In [363]:
np.where(np.concatenate(events.LptElectron.IDscore) < 0)

(<Array [3, 6, 8, 16, ... 17623, 17625, 17628] type='2135 * int64'>,)

In [ ]:
l=0
j=0
while l < 4202:
    while j < len(events.LptElectron.IDscore):
        if l == 4201:
            print(j)
        j += 1
        if len(events.LptElectron.IDscore[j]) > 0:
            for k in range(len(events.LptElectron.IDscore[j])):
                l += 1

In [239]:
events.Electron.IDscore[i]

<Array [0, 0] type='2 * float32[parameters={"__doc__": "Electron_IDmvaLoose"}]'>

In [240]:
events.LptElectron.IDscore[i]

<Array [0.282, 0.814] type='2 * float32[parameters={"__doc__": "LptElectron_ID"}]'>

In [125]:
events.vtx.e1[5]

<Array [Electron, ... passID: True}] type='3 * union[electron, struct[["pt", "et...'>

In [126]:
events.vtx.e2[5]

<Array [Electron, Electron, Electron] type='3 * union[electron, struct[["pt", "e...'>

In [243]:
events.LptElectron[i]

<NanoCollectionArray [{pt: 27.1, ... passID: False}] type='2 * struct[["pt", "et...'>

In [244]:
events.Electron[i]

<ElectronArray [Electron, Electron] type='2 * electron'>